## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('./data/ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="./data/ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('./data/ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])
print(sentences_ted[2])
print(sentences_ted[3])
print(sentences_ted[4])


['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
['consider', 'facit']
['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
from collections import Counter
word_counts = Counter()
for sentence in sentences_ted:
    for word in sentence:
        word_counts[word] += 1

In [14]:
word_counts.most_common(1000)

[('the', 207748),
 ('and', 149305),
 ('to', 125169),
 ('of', 114818),
 ('a', 105399),
 ('that', 95146),
 ('i', 83180),
 ('in', 78070),
 ('it', 74738),
 ('you', 70923),
 ('we', 67629),
 ('is', 63251),
 ('s', 57156),
 ('this', 49241),
 ('so', 37014),
 ('they', 33102),
 ('was', 30806),
 ('for', 29713),
 ('are', 27995),
 ('have', 27344),
 ('but', 26732),
 ('what', 26519),
 ('on', 25962),
 ('with', 24706),
 ('can', 23377),
 ('t', 22757),
 ('about', 21246),
 ('there', 21041),
 ('be', 20201),
 ('as', 19488),
 ('at', 19216),
 ('all', 19021),
 ('not', 18626),
 ('do', 17928),
 ('my', 17908),
 ('one', 17551),
 ('re', 17012),
 ('people', 16723),
 ('like', 16273),
 ('if', 15868),
 ('from', 15452),
 ('now', 14387),
 ('our', 14061),
 ('he', 13986),
 ('an', 13917),
 ('just', 13896),
 ('these', 13882),
 ('or', 13864),
 ('when', 13278),
 ('because', 12879),
 ('very', 12363),
 ('me', 12302),
 ('out', 12163),
 ('by', 11935),
 ('them', 11595),
 ('how', 11576),
 ('know', 11506),
 ('up', 11445),
 ('going', 1

In [15]:
counts_ted_top1000 = [x[1] for x in word_counts.most_common(1000)]

Plot distribution of top-1000 words

In [16]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [17]:
from gensim.models import Word2Vec

In [18]:
?Word2Vec

In [19]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [20]:
model_ted.most_similar("man")

[('woman', 0.8464012145996094),
 ('guy', 0.8135526180267334),
 ('lady', 0.7727985382080078),
 ('boy', 0.7620566487312317),
 ('girl', 0.7531026601791382),
 ('gentleman', 0.7288511991500854),
 ('soldier', 0.7221850752830505),
 ('poet', 0.6835501194000244),
 ('kid', 0.6622081995010376),
 ('philosopher', 0.6593121886253357)]

In [21]:
model_ted.most_similar("computer")

[('machine', 0.7324864268302917),
 ('software', 0.7034176588058472),
 ('robot', 0.6967039704322815),
 ('3d', 0.6636025905609131),
 ('device', 0.6527997255325317),
 ('video', 0.6352449655532837),
 ('program', 0.6321359872817993),
 ('mouse', 0.6216418147087097),
 ('camera', 0.6208375692367554),
 ('code', 0.6106781363487244)]

In [22]:
model_ted.most_similar('india')

[('america', 0.8837741017341614),
 ('europe', 0.8801760673522949),
 ('china', 0.8712195158004761),
 ('brazil', 0.8628115057945251),
 ('kenya', 0.8599361181259155),
 ('africa', 0.8580676913261414),
 ('australia', 0.8559602499008179),
 ('germany', 0.8451232314109802),
 ('afghanistan', 0.8433178663253784),
 ('mexico', 0.8423097133636475)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [23]:
words_top_ted = [x[0] for x in word_counts.most_common(1000)]

In [24]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [25]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [26]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [27]:
if not os.path.isfile('./data/wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="./data/wikitext-103-raw-v1.zip")

In [28]:
with zipfile.ZipFile('./data/wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [29]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [30]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [31]:
sentences_wiki[0:5]

[' all of the captured men were then marched to farnham tied together in pairs ',
 ' the architectural style is a mix of renaissance revival and romanesque ',
 '        million contract by the giants ',
 '   in ridgeley   west virginia   as trucks became stuck under a low railroad overpass   blocking traffic through ridgeley ',
 '         d   is the fourth episode of the ninth season and the    th episode overall of the american science fiction television series the x     files ']

In [32]:
sentences_wiki_splitted = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_splitted.append(tokens)

In [33]:
sentences_wiki_splitted[0:1]

[['all',
  'of',
  'the',
  'captured',
  'men',
  'were',
  'then',
  'marched',
  'to',
  'farnham',
  'tied',
  'together',
  'in',
  'pairs']]

In [34]:
word_counts_wiki = Counter()
for sentence in sentences_wiki_splitted:
    for word in sentence:
        word_counts_wiki[word] += 1

In [35]:
word_counts_wiki.most_common(1000)

[('the', 1286419),
 ('of', 547900),
 ('and', 501017),
 ('in', 434913),
 ('to', 398646),
 ('a', 357876),
 ('was', 215561),
 ('s', 163797),
 ('on', 154610),
 ('as', 145034),
 ('for', 139624),
 ('that', 138369),
 ('with', 129991),
 ('by', 126408),
 ('is', 102925),
 ('his', 92860),
 ('at', 92432),
 ('he', 91366),
 ('it', 89190),
 ('from', 88922),
 ('were', 70737),
 ('an', 64549),
 ('had', 58824),
 ('which', 55025),
 ('be', 47674),
 ('this', 43259),
 ('but', 42502),
 ('first', 41106),
 ('are', 41051),
 ('not', 39775),
 ('their', 38710),
 ('after', 38217),
 ('one', 37746),
 ('her', 37320),
 ('also', 37147),
 ('its', 36681),
 ('they', 34915),
 ('two', 34358),
 ('or', 33375),
 ('have', 33132),
 ('who', 32147),
 ('been', 30853),
 ('has', 30173),
 ('new', 30161),
 ('she', 29611),
 ('when', 27245),
 ('during', 26573),
 ('time', 26491),
 ('other', 25814),
 ('would', 25643),
 ('into', 24793),
 ('all', 24371),
 ('i', 23920),
 ('more', 23192),
 ('while', 21456),
 ('over', 20842),
 ('later', 19900),
 

In [36]:
model_wiki = Word2Vec(sentences_wiki_splitted, min_count=10)

In [37]:
words_top_wiki = [x[0] for x in word_counts_wiki.most_common(1000)]

In [38]:
model_wiki.most_similar("man")

[('woman', 0.7149965763092041),
 ('boy', 0.6484652161598206),
 ('girl', 0.6334134936332703),
 ('person', 0.5860021114349365),
 ('soldier', 0.5751364827156067),
 ('dog', 0.5518984794616699),
 ('men', 0.5463042259216309),
 ('creature', 0.5397509932518005),
 ('mask', 0.5252947211265564),
 ('demon', 0.5205068588256836)]

In [39]:
model_wiki.most_similar("computer")

[('software', 0.753383219242096),
 ('handheld', 0.723432183265686),
 ('hardware', 0.7128636837005615),
 ('computers', 0.711846649646759),
 ('desktop', 0.6946860551834106),
 ('interface', 0.6895376443862915),
 ('user', 0.6806899309158325),
 ('virtual', 0.6800329685211182),
 ('interactive', 0.6762830018997192),
 ('simulation', 0.6735183000564575)]

In [40]:
model_wiki.most_similar("india")

[('africa', 0.7591451406478882),
 ('china', 0.7458372116088867),
 ('indonesia', 0.7432065010070801),
 ('pakistan', 0.7340015172958374),
 ('australia', 0.7021082639694214),
 ('asia', 0.6673640012741089),
 ('malaysia', 0.6560901999473572),
 ('europe', 0.6536030173301697),
 ('singapore', 0.6523457765579224),
 ('korea', 0.6428737044334412)]

#### t-SNE visualization

In [41]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [42]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)